In [4]:
import databento as db
import pandas as pd
from datetime import datetime

# Initialize Data Bento client
client = db.Historical("db-hLqXJHUbgC3mVyJ9T6mEjVwgN57f4")

# Example tickers and date range
tickers = ['SPY']
start_date = '2020-01-01'
end_date = '2024-09-01'

for ticker in tickers:
    dataset = client.timeseries.get_range(
        dataset="XNAS.ITCH",
        symbols=ticker,
        start=start_date,
        end=end_date,
        schema='ohlcv-1d'
    )
    dataset.to_csv(f'{ticker}_data.csv')


AttributeError: partially initialized module 'databento' has no attribute 'Historical' (most likely due to a circular import)

#### Combined file, downloads then converts using databento

In [ ]:
import pandas as pd
def convert_to_lean_format(csv_file, ticker, frequency='daily'):
    """
    Converts a CSV file containing stock data into a format compatible with LEAN Local CLI Framework.
    Args:
        csv_file (str): The path to the input CSV file containing stock data.
        ticker (str): The stock ticker symbol.
        frequency (str, optional): The frequency of the data. Can be 'daily', 'hourly', or 'minute'. Defaults to 'daily'.
    Returns:
        None: The function saves the converted data to a file in the appropriate directory based on the frequency. Located within project directory.
    """
    
    df = pd.read_csv(csv_file)
    df['date'] = pd.to_datetime(df.pop('ts_event')).dt.strftime('%Y%m%d')
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    
    if frequency == 'daily':
        output_file = f"data/equity/usa/daily/{ticker}.csv"
    elif frequency == 'hourly':
        output_file = f"data/equity/usa/hourly/{ticker}.csv"
    else:
        output_file = f"data/equity/usa/minute/{ticker}.zip"
    
    df.to_csv(output_file, index=False)

def download_and_convert(tickers, start_date, end_date, frequency='daily'):
    for ticker in tickers:
        dataset = client.timeseries.get_range(
            dataset="XNAS.ITCH",
            symbols=ticker,
            start=start_date,
            end=end_date,
            schema='ohlcv-1d'
        )
        csv_file = f'{ticker}_data.csv'
        dataset.to_csv(csv_file)

        # Convert to QuantConnect format
        convert_to_lean_format(csv_file, ticker, frequency)

# Example ticker list and date range
ticker_list = ['AAPL', 'GOOG', 'MSFT']
download_and_convert(ticker_list, '2023-01-01', '2023-09-01', 'daily')
